In [1]:
# Imports
import sys
from pathlib import Path
from typing import Optional
import time


def find_project_root(
    start: Path = Path.cwd(), marker: str = "pyproject.toml"
) -> Optional[Path]:
    for p in [start] + list(start.parents):
        if (p / marker).is_file():
            return p
    return None


PROJECT_ROOT: Optional[Path] = find_project_root()
if PROJECT_ROOT:
    proj_path = str(PROJECT_ROOT)
    if proj_path not in sys.path:
        sys.path.insert(0, proj_path)
else:
    print(
        f"Warning: pyproject.toml not found in parents of {Path.cwd()}; sys.path unchanged"
    )

In [2]:
from src.servo.classes_servo import ServoController

In [3]:
# Create the servo controller for testing/demo
servo_controller: ServoController = ServoController(channel=0)

PCA9685 initialized successfully


In [4]:
target_angle: float = 0.0
speed_deg_per_sec: float = 60.0  # degrees per second

In [40]:
servo_controller.move_to_center(speed_deg_per_sec=30.0)

In [39]:
servo_controller._drv_pwm.set_pwm(
        0, 0, 145
    )

In [8]:
for current_counts in range(307, 0, -1):

    print(f"Setting servo to counts: {current_counts}")
    servo_controller._drv_pwm.set_pwm(
        0, 0, current_counts
    )

    time.sleep(0.5)

Setting servo to counts: 307
Setting servo to counts: 306
Setting servo to counts: 305
Setting servo to counts: 304
Setting servo to counts: 303
Setting servo to counts: 302
Setting servo to counts: 301
Setting servo to counts: 300
Setting servo to counts: 299
Setting servo to counts: 298
Setting servo to counts: 297
Setting servo to counts: 296
Setting servo to counts: 295
Setting servo to counts: 294
Setting servo to counts: 293
Setting servo to counts: 292
Setting servo to counts: 291
Setting servo to counts: 290
Setting servo to counts: 289
Setting servo to counts: 288
Setting servo to counts: 287
Setting servo to counts: 286
Setting servo to counts: 285
Setting servo to counts: 284
Setting servo to counts: 283
Setting servo to counts: 282
Setting servo to counts: 281
Setting servo to counts: 280
Setting servo to counts: 279
Setting servo to counts: 278
Setting servo to counts: 277
Setting servo to counts: 276
Setting servo to counts: 275
Setting servo to counts: 274
Setting servo 

KeyboardInterrupt: 

In [ ]:
servo_controller.move_by_angle(-90.0, speed_deg_per_sec=30.0)

In [ ]:
for i in range(5):
    servo_controller.move_to(target_angle, speed_deg_per_sec=speed_deg_per_sec)
    servo_controller.move_to_center(speed_deg_per_sec=speed_deg_per_sec)